<a href="https://colab.research.google.com/github/CVStack/2020_2-AI/blob/master/CNN_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1 , cache = True)
X = mnist.data
y = mnist.target

import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/7, random_state = 0)
#split train, test data
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.LongTensor(list(map(int, y_train)))
y_test = torch.LongTensor(list(map(int, y_test)))

import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable

X_train = X_train.view(-1, 1, 28, 28).float()
X_test = X_test.view(-1, 1, 28, 28).float()
print(X_train.shape)
print(X_test.shape)

train = TensorDataset(X_train, y_train)
test = TensorDataset(X_test, y_test)
BATCH_SIZE = 32
loader_train = DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
loader_test = DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size = 5) # 1 -> feature map 32 // filter : 32
    self.conv2 = nn.Conv2d(32, 32, kernel_size = 5) # 32 -> 32 // filter : 1
    self.conv3 = nn.Conv2d(32, 64, kernel_size = 5) # 32 -> 64 // fiter : 2
    self.fc1 = nn.Linear(3*3*64, 256)
    self.fc2 = nn.Linear(256, 10)

    self.loss_fn = nn.CrossEntropyLoss()
    self.optimizer = optim.Adam(self.parameters(), lr = 0.01)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(F.max_pool2d(self.conv2(x), 2))
    x = F.dropout(x, p = 0.5, training=self.training)
    x = F.relu(F.max_pool2d(self.conv3(x) ,2))
    x = F.dropout(x, p = 0.5, training=self.training)
    x = x.view(-1, 3*3*64)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return F.log_softmax(x, dim = 1)

  

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])


In [ ]:
def fit(model, loader_train):
    optimizer = torch.optim.Adam(model.parameters())
    error = nn.CrossEntropyLoss()
    EPOCHS = 1
    model.train()
    for epoch in range(EPOCHS):
      correct = 0
      for batch_idx, (X_batch, y_batch) in enumerate(loader_train):
        var_X_batch = Variable(X_batch).float()
        var_y_batch = Variable(y_batch)
        optimizer.zero_grad()
        output = model(var_X_batch)
        loss = error(output, var_y_batch)
        loss.backward()
        optimizer.step() #parameter update
        predicted = torch.max(output.data , 1)[1]
        correct += (predicted == var_y_batch).sum()
        if batch_idx % 50 == 0:
          print('epoch : {}[{}/{} ({:.0f})]\t loss function : {:.6f}\t Accuracy:{:.3f}%'
          .format(epoch, batch_idx*len(X_batch), len(loader_train), 
                  100. * batch_idx / len(loader_train), loss.data,
                  correct * 100./ (BATCH_SIZE * (batch_idx + 1))))
          
def evaluate(model):
    correct = 0
    for test_imgs, test_labels in loader_test:
      test_imgs = Variable(test_imgs).float()
      outputs = model(test_imgs)
      predicted = torch.max(outputs, 1)[1]
      correct += (predicted == test_labels).sum()
    print('test data accuracy : {:.3f}%'.format(float(correct) / (len(loader_test) * BATCH_SIZE)))

cnn = CNN()
evaluate(cnn)
fit(cnn, loader_train)
cnn.eval()
evaluate(cnn)

index = 10
data = X_test[index].view(-1, 1, 28, 28).float()
output = cnn(data)
print('{}번째 학습데이터의 테스트 결과 : {}'.format(index, output))
_, predicted = torch.max(output, 1)
print('{}번째 데이터 예측 : {}'.format(index, predicted.numpy()))
print('실제 레이블 : {}'.format(y_test[index]))

test data accuracy : 0.100%
epoch : 0[0/1875 (0)]	 loss function : 13.764584	 Accuracy:12.500%
epoch : 0[1600/1875 (3)]	 loss function : 2.100624	 Accuracy:17.463%
epoch : 0[3200/1875 (5)]	 loss function : 1.063302	 Accuracy:35.520%
epoch : 0[4800/1875 (8)]	 loss function : 0.875107	 Accuracy:48.262%
epoch : 0[6400/1875 (11)]	 loss function : 0.834617	 Accuracy:55.442%
epoch : 0[8000/1875 (13)]	 loss function : 0.517701	 Accuracy:61.056%
epoch : 0[9600/1875 (16)]	 loss function : 0.303024	 Accuracy:65.293%
epoch : 0[11200/1875 (19)]	 loss function : 0.317802	 Accuracy:68.225%
epoch : 0[12800/1875 (21)]	 loss function : 0.447770	 Accuracy:70.511%
epoch : 0[14400/1875 (24)]	 loss function : 0.380069	 Accuracy:72.457%
epoch : 0[16000/1875 (27)]	 loss function : 0.228485	 Accuracy:74.071%
epoch : 0[17600/1875 (29)]	 loss function : 0.183773	 Accuracy:75.482%
epoch : 0[19200/1875 (32)]	 loss function : 0.587907	 Accuracy:76.560%
epoch : 0[20800/1875 (35)]	 loss function : 0.486716	 Accuracy